# Forming features 

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os
import sys
from geopy.geocoders import Nominatim
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.inspection import permutation_importance
from datetime import datetime, timedelta

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from datetime import timedelta

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.multioutput import MultiOutputRegressor

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append(os.path.dirname(os.getcwd()))
from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR, PROJ_ROOT, INTERIM_DATA_DIR
from scrpts.data_from_stations import get_weather_data
from scrpts.data_from_stations import get_weather_data_timeInterval

from scrpts.data_from_stations import get_air_pollution_data_timeInterval

from scrpts.data_from_stations import get_air_pollution_data
from scrpts.wfs import get_stored_query_descriptions

In [4]:
air_pollution_stations=['Helsinki Kallio 2',
 'Espoo Leppävaara Läkkisepänkuja',
 'Espoo Luukki',
 'Helsinki Mannerheimintie',
 'Vantaa Tikkurila Neilikkatie',
 'Helsinki Vartiokylä Huivipolku',
 'Vantaa Kehä III Viinikkala',
 'Helsinki Kustaa Vaasan tie']

weather_stations= [
 'Helsinki Vuosaari satama',
 'Helsinki Kaisaniemi',
 'Helsinki Kumpula',
 'Helsinki Malmi lentokenttä',
 'Sipoo Itätoukki']

In [5]:
def plot_prediction(y_test_original, predicted_concentration_original, start_date):
    """
    Plot the real and predicted values for courier partners online.

    Parameters:
    y_test_original (array-like): The original test values (real values).
    predicted_courier_number_original (array-like): The predicted values from the model.

    Returns:
    None
    """

    print(y_test_original)
    print(predicted_concentration_original)
    
    date_range = pd.date_range(start=start_date, periods=len(y_test_original))
    plt.plot(date_range, y_test_original, color='red', marker='+', label="Real")
    plt.plot(date_range, predicted_concentration_original, color="blue", marker='o', label="Predicted")
    plt.title("Prediction")
    plt.xlabel("Day")
    plt.ylabel("Number of Courier Partners Online")
    plt.legend()
    plt.grid()
    plt.xticks(rotation=45)
    plt.show()


forming database for air pollution

In [6]:
#df_weather_total = pd.read_csv(os.path.join(INTERIM_DATA_DIR, 'weather_data_total.csv'))
df_air_pollution_total = pd.read_parquet(os.path.join(INTERIM_DATA_DIR, 'air_pollution_data_total.parquet'))

In [7]:
df_air_pollution_total['Timestamp'] = pd.to_datetime(df_air_pollution_total['Timestamp'])

In [8]:
df_air_pollution_total.head()

,Timestamp,Particulate matter < 10 µm_Helsinki Kallio 2,Particulate matter < 2.5 µm_Helsinki Kallio 2,Nitrogen dioxide_Helsinki Kallio 2,Particulate matter < 10 µm_Espoo Leppävaara Läkkisepänkuja,Particulate matter < 2.5 µm_Espoo Leppävaara Läkkisepänkuja,Nitrogen dioxide_Espoo Leppävaara Läkkisepänkuja,Particulate matter < 10 µm_Espoo Luukki,Particulate matter < 2.5 µm_Espoo Luukki,Nitrogen dioxide_Espoo Luukki,...,Nitrogen dioxide_Vantaa Tikkurila Neilikkatie,Particulate matter < 10 µm_Helsinki Vartiokylä Huivipolku,Particulate matter < 2.5 µm_Helsinki Vartiokylä Huivipolku,Nitrogen dioxide_Helsinki Vartiokylä Huivipolku,Particulate matter < 10 µm_Vantaa Kehä III Viinikkala,Particulate matter < 2.5 µm_Vantaa Kehä III Viinikkala,Nitrogen dioxide_Vantaa Kehä III Viinikkala,Particulate matter < 10 µm_Helsinki Kustaa Vaasan tie,Particulate matter < 2.5 µm_Helsinki Kustaa Vaasan tie,Nitrogen dioxide_Helsinki Kustaa Vaasan tie
0,2025-05-16 17:00:00,10.6,4.3,7.0,10.2,3.6,3.3,6.5,2.5,0.1,...,9.4,8.7,3.7,1.2,51.8,8.8,3.3,10.8,3.9,10.1
1,2025-05-16 18:00:00,11.3,4.7,6.1,10.8,3.6,4.2,7.3,2.6,0.1,...,10.9,8.2,4.0,15.2,54.7,8.9,3.6,11.4,4.8,10.1
2,2025-05-16 19:00:00,11.7,4.8,7.0,13.0,4.5,7.7,6.2,2.7,0.1,...,12.6,10.0,4.6,8.8,16.6,4.0,3.7,12.0,5.9,9.3
3,2025-05-16 20:00:00,6.4,3.2,13.4,11.1,5.1,22.4,8.9,4.1,5.4,...,16.0,6.8,3.7,6.9,10.9,4.9,11.5,8.4,4.6,20.9
4,2025-05-16 21:00:00,3.4,1.2,5.2,8.6,3.2,7.7,7.6,4.2,7.1,...,10.5,3.9,1.4,17.7,8.5,4.1,13.3,6.2,1.8,11.6


In [ ]:
df_air_pollution_kallio = df_air_pollution_total[['Timestamp', 
                                                  'Particulate matter < 10 µm_Helsinki Kallio 2', 
                                                  'Particulate matter < 2.5 µm_Helsinki Kallio 2',
                                                  'Nitrogen dioxide_Helsinki Kallio 2']].copy()

In [61]:
features_pollution = list(df_air_pollution_total.columns[1:])
features_pollution = [col for col in features_pollution if 'matter' in col]

#features = list(df_air_pollution_total.columns[1:])
features_additional = []
features_additional.append('hour_sin')
features_additional.append('hour_cos')
features_additional.append('day_sin')      
features_additional.append('day_cos')

hours = df_air_pollution_total['Timestamp'].dt.hour
days_of_week = df_air_pollution_total['Timestamp'].dt.dayofweek  # Monday=0, Sunday=6

    # Hour of day (24 hours)
df_air_pollution_total['hour_sin'] = np.sin(2 * np.pi * hours / 24)
df_air_pollution_total['hour_cos'] = np.cos(2 * np.pi * hours / 24)

    # Day of week (7 days)
df_air_pollution_total['day_sin'] = np.sin(2 * np.pi * days_of_week / 7)
df_air_pollution_total['day_cos'] = np.cos(2 * np.pi * days_of_week / 7)

# Get the latest time from the datasets
latest_time = df_air_pollution_total['Timestamp'].max()
earliest_time = df_air_pollution_total['Timestamp'].min()

set_pollution_additional = np.array( df_air_pollution_total[features_additional ].values)
set_pollution_target = np.array( df_air_pollution_total[features_pollution ].values)

# Print confirmation
print("Data successfully split and scaled.")

Data successfully split and scaled.


In [62]:
#df_total = pd.merge(df_air_pollution_total, df_weather_total, how='outer', on='Timestamp')
#df_total.dropna(inplace=True)

In [63]:
training_hours = 24
n_steps = 6

In [ ]:
# Prepare the training data
X = []
X_additional = []
y = []

for i in range(training_hours, len(set_pollution_additional) - n_steps + 1):

    #print(set_pollution_additional[i])
    #print(set_pollution_target[i - training_hours:i])    
 
    X.append(set_pollution_target[i - training_hours:i])
    X_additional.append(set_pollution_additional[i])

    y.append(set_pollution_target[i:i + n_steps])

X, X_additional, y = np.array(X), np.array(X_additional), np.array(y)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
#X_additional = X_additional.reshape(X_additional.shape[0], X_additional.shape[1] * X_additional.shape[2])
y = y.reshape(y.shape[0], y.shape[1] * y.shape[2])

X = np.concatenate((X, X_additional), axis=1)

X shape: (1315, 24, 16)
y shape: (1315, 6, 16)


In [65]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

tscv = TimeSeriesSplit(n_splits=3)

# Get the last split for validation
for train_index, val_index in tscv.split(X):
    pass  # After the loop, train_index and val_index will be from the last split

X_train, X_val = X[train_index], X[val_index]
y_train, y_val = y[train_index], y[val_index]

SC = StandardScaler()
X_train = SC.fit_transform(X_train)
X_val = SC.transform(X_val)
X = SC.transform(X)

In [66]:
#if n_steps == 1:
#    y_train = y_train.flatten()  # Single-output regression
#    X_train = X_train.flatten()   # Flatten the features
#    LRmodel = LinearRegression()
#else:
    
LRmodel = MultiOutputRegressor(LinearRegression())  # Multi-step regression

LRmodel.fit(X_train, y_train)

,estimator,LinearRegression()
,n_jobs,None
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [67]:
# Ridge Regression
ridge_model = MultiOutputRegressor(Ridge(alpha=0.10)) # Adjust alpha
ridge_model.fit(X_train, y_train)
ridge_predictions = ridge_model.predict(X_val)

In [68]:
# Lasso Regression
lasso_model = MultiOutputRegressor(Lasso(alpha=1.0)) # Adjust alpha
lasso_model.fit(X_train, y_train)
lasso_predictions = lasso_model.predict(X_val)

In [69]:
# Elastic Net Regression
elastic_net_model = MultiOutputRegressor(ElasticNet(alpha=0.1, l1_ratio=0.5)) # Adjust alpha and l1_ratio
elastic_net_model.fit(X_train, y_train)
elastic_net_predictions = elastic_net_model.predict(X_val)

In [70]:
#rf_reg = RandomForestRegressor(
#    n_estimators=100,     # Number of trees in the forest
#    max_depth=None,       # Maximum depth of the trees
#    random_state=42,      # For reproducibility
#    n_jobs=-1             # Use all available cores for training
#)
#RFmodel = MultiOutputRegressor(rf_reg) 

#RFmodel.fit(X_train, y_train)

In [71]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgb_reg = XGBRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)
XGBmodel = MultiOutputRegressor(xgb_reg)

XGBmodel.fit(X_train, y_train)

,estimator,"XGBRegressor(...ree=None, ...)"
,n_jobs,None
,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None


In [72]:
accuracy = XGBmodel.score(X_val, y_val)
print(f"XGB accuracy: {accuracy:.2f}")

XGB accuracy: 0.15


In [74]:
#ccuracy = RFmodel.score(X_val, y_val)
#print(f"RF accuracy: {accuracy:.2f}")

In [75]:
accuracy = ridge_model.score(X_val, y_val)
print(f"Ridge Model accuracy: {accuracy:.2f}")

Ridge Model accuracy: -0.21


In [76]:
accuracy = lasso_model.score(X_val, y_val)
print(f"Lasso Model accuracy: {accuracy:.2f}")

Lasso Model accuracy: 0.27


In [77]:
accuracy = elastic_net_model.score(X_val, y_val)
print(f"Elastic Model accuracy: {accuracy:.2f}")

Elastic Model accuracy: 0.25


In [78]:
accuracy = LRmodel.score(X_val, y_val)
print(f"LR Model accuracy: {accuracy:.2f}")

LR Model accuracy: -0.22


In [80]:
#latest_prediction = RFmodel.predict(X)

In [81]:
latest_prediction = latest_prediction.reshape(669, 2, 16)

In [82]:
latest_prediction[-1,:,0]

array([5.401, 6.554])

In [ ]:
latest_prediction[-1,:,1]

array([3.034, 3.528])

In [ ]:
set_pollution_target[-1]

array([4.9, 2.9, 6.1, 3.5, 4.8, 2.7, 7.3, 4.4, 6.3, 3.7, 5.4, 3.3, 9.6,
       5. , 8.4, 4.5])